In [ ]:
import requests
import pandas as pd
import sqlite3
import os
from dotenv import load_dotenv
load_dotenv()

dbconn = os.getenv("DBCONN")
api_key = os.getenv("API_KEY")
print(api_key)

url = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey=" + api_key

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


In [ ]:
status_code = response.status_code
data = response.json()
print(status_code)
data

In [ ]:
import csv
from datetime import datetime, timedelta

def extract_and_save_to_csv(data, filename="filtered_data.csv"):
    """
    Extracts daily cryptocurrency data for the last 6 months and saves it to a CSV file.

    Parameters:
        data (dict): JSON-like dictionary containing Meta Data and Time Series data.
        filename (str): Name of the CSV file to save the data.
    
    Returns:
        None
    """
    # Get the last refreshed date from metadata
    last_refreshed_str = data['Meta Data']['6. Last Refreshed']
    last_refreshed = datetime.strptime(last_refreshed_str, "%Y-%m-%d %H:%M:%S")

    # Calculate the date 6 months ago
    six_months_ago = last_refreshed - timedelta(days=6 * 30)  # Approximate 6 months

    # Filter data for the last 6 months
    filtered_data = {
        date: values
        for date, values in data['Time Series (Digital Currency Daily)'].items()
        if datetime.strptime(date, "%Y-%m-%d") >= six_months_ago
    }

    # Define CSV column headers
    headers = ["Date", "Open", "High", "Low", "Close", "Volume"]

    # Save the filtered data to a CSV file
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write header row

        for date, values in filtered_data.items():
            writer.writerow([
                date,
                values["1. open"],
                values["2. high"],
                values["3. low"],
                values["4. close"],
                values["5. volume"]
            ])
    
    print(f"✅ Data successfully saved to {filename}")

# Example usage
extract_and_save_to_csv(data, "last_6_months_data.csv")


In [ ]:


# Connecting to the database with a timeout to handle locks
Bitcoin_connection = sqlite3.connect("Bitcoin.db", timeout=10)
crsr = Bitcoin_connection.cursor()

# Set busy timeout and WAL mode
crsr.execute("PRAGMA busy_timeout = 3000;")
print("Connected to the database")

# Load the CSV
Bitcoin_csv = pd.read_csv(
    r"/Users/jaamann/Documents/Vs Code Python /Code Academy/Projects/5 Data Pipeline Api & web scraping/last_6_months_data.csv",
    sep=",",
    encoding='latin1',
    on_bad_lines='skip'
)
# Create Customer table
crsr.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin (
    "Date" TEXT PRIMARY KEY,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Volume FLOAT
);
''')
print("Bitcoin table created successfully")
Bitcoin_connection.commit()  # Save changes


In [ ]:
# Ensure column names match the database schema
Bitcoin_csv.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]

# Insert data into the database
for index, row in Bitcoin_csv.iterrows():
    crsr.execute('''
        INSERT OR IGNORE INTO Bitcoin ("Date", Open, High, Low, Close, Volume)
        VALUES (?, ?, ?, ?, ?, ?);
    ''', (row["Date"], row["Open"], row["High"], row["Low"], row["Close"], row["Volume"]))

# Commit changes and close connection
Bitcoin_connection.commit()
Bitcoin_connection.close()

print("✅ Data successfully inserted into Bitcoin.db")


In [ ]:
Bitcoin_csv.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
Bitcoin_csv = pd.read_csv(
    r"/Users/jaamann/Documents/Vs Code Python /Code Academy/Projects/5 Data Pipeline Api & web scraping/last_6_months_data.csv",
    sep=",",
    encoding='latin1',
    on_bad_lines='skip'
)

# Convert "Date" to a datetime object for proper plotting
Bitcoin_csv["Date"] = pd.to_datetime(Bitcoin_csv["Date"])

# Sort data by date (optional, but good for plotting)
Bitcoin_csv = Bitcoin_csv.sort_values("Date")

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(Bitcoin_csv["Date"], Bitcoin_csv["Open"], marker='o', linestyle='-', color='g', label="Opening Price")
plt.plot(Bitcoin_csv["Date"], Bitcoin_csv["Close"], marker='o', linestyle='-', color='b', label="Closing Price")

# Formatting the plot
plt.xlabel("Date")
plt.ylabel("Bitcoin Price (EUR)")
plt.title("Bitcoin Opening vs Closing Prices Over Time")
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()

# Show the plot
plt.show()
